In [28]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,KBinsDiscretizer,MinMaxScaler,StandardScaler,PolynomialFeatures

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression

from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict

from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import precision_recall_curve,roc_curve

from sklearn.neighbors import KNeighborsClassifier

from sklearn.decomposition import KernelPCA,PCA

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
df_train = pd.read_csv('train.csv',na_values=[' '])
#display(df_train.NOEXCH.unique())
#df_train.head()
#df_test = pd.read_csv('train.csv')

df_train.loc[df_train.NOEXCH == 1, 'NOEXCH']   = 'X'
df_train.loc[df_train.NOEXCH == 0, 'NOEXCH']   = np.nan
df_train.loc[df_train.NOEXCH == '1', 'NOEXCH']   = 'X'
df_train.loc[df_train.NOEXCH == '0', 'NOEXCH']   = np.nan

df_train.NOEXCH.unique()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,8,9,10,11,12,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([nan, 'X'], dtype=object)

In [ ]:
#print(df_train.isna().sum())
#df_train.describe().T

In [ ]:
# features1= [] 
# #list(['cat1','cat2','cat3'])

# for i in range(3,25):
#     features1.append('RAMNT_' + str(i) )
# features1.append()
# df_train[features1].corr()


In [ ]:
# print(df_train[['RAMNTALL']].isna().sum())
# df_train[['RAMNTALL']].apply(lambda x: x.corr(df_train.TARGET_D))

# #df_train.drop("TARGET_B", axis=1).apply(lambda x: x.corr(df_train.TARGET_B))

In [ ]:

c = df_train.corr().abs()

s = c.unstack()

so = s.sort_values(kind="quicksort")

display(so.shape)

print(so[-2000:-1])

In [ ]:
df_train.corr()['TARGET_B']
# cols=df_train.columns[(df_train.isna().sum() == 0)]
# cols = cols[cols != 'OSOURCE']
# cols = cols[cols != 'STATE']
# display(cols)

# df_new[cols].apply(lambda x: x.corr(df_new.TARGET_B))

## Pre processing

In [ ]:
imputer = SimpleImputer(strategy = 'most_frequent')
df_train[['AGE']] = imputer.fit_transform(df_train[['AGE']])



## set X,y

In [ ]:
X = df_train.drop(['TARGET_B','TARGET_D'],axis=1)
y = df_train.TARGET_B

X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
#X_train,X_test ,y_train ,y_test

#print(X_train.isna().sum().to_string())

In [ ]:
X_train_sub = X_train[['AGE']]
X_test_sub = X_test[['AGE']]



## LogisticRegression

In [ ]:
model = LogisticRegression()
model.fit(X_train_sub,y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(model,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)

## SGDClassifier

In [ ]:
sgd_clf = SGDClassifier(random_state=0,max_iter=50)
sgd_clf.fit(X_train_sub,y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(sgd_clf,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(sgd_clf.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)


### RandomForestClassifier

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=1)
forest_clf.fit(X_train_sub,y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(forest_clf,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(forest_clf.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)

### LogisticRegression + PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=4)

model = LogisticRegression(solver='lbfgs', multi_class='multinomial')
model = make_pipeline(poly, model)
model.fit(X_train_sub, y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(model,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)
    


### SGDClassifier + PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=4)

sgd_clf = SGDClassifier(random_state=0,max_iter=50)
sgd_clf = make_pipeline(poly, sgd_clf)
sgd_clf.fit(X_train_sub, y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(sgd_clf,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(sgd_clf.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)
    


### KNeighborsClassifier

In [ ]:
poly = PolynomialFeatures(degree=4)

knn = make_pipeline(poly, KNeighborsClassifier(n_neighbors=5))
knn.fit(X_train_sub, y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(knn,X_test_sub,y_test,cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(knn.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)

In [36]:
#print(X_train.isna().sum())
# X_train.info()
# print(X_train.select_dtypes(include=['float64','int64']).columns)
X = df_train.drop(['TARGET_B','TARGET_D'],axis=1)
y = df_train.TARGET_B

print(X.shape)

numeric_features = X.select_dtypes(include=['float64','int64']).columns
numeric_transformer = Pipeline([('imputer',SimpleImputer(strategy='median'))])

#create the preprocessing pipeline for the categorial data
category_features = X.select_dtypes(include=['object']).columns
category_transformer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
                               ('onehot',OneHotEncoder())])


preprocessor = ColumnTransformer([
     ('num',numeric_transformer,numeric_features)
#     ('cat',category_transformer,category_features)
])#,remainder='passthrough')

clf = Pipeline([
    ('pre',preprocessor)
    #('cls',RandomForestClassifier(n_estimators=5))
])



#X_train,X_test,y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
#X_train,X_test ,y_train ,y_test

X_train_pre = clf.fit_transform(X)




(95412, 479)


In [37]:
df = pd.DataFrame(X_train_pre)
df
#print(df.isna().sum())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410
0,9001.0,1.0,1509.0,28.0,82.0,1.0,4.0,9.0,19.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,9.0,5.0,2.0,545.0,165.0,213.0,99.0,0.0,0.0,51.0,49.0,94.0,1.0,0.0,4.0,8.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,5.0,0.0,0.0,3.0,37.0,43.0,44.0,39.0,48.0,49.0,21.0,46.0,34.0,20.0,5.0,24.0,24.0,14.0,12.0,14.0,8.0,17.0,25.0,31.0,18.0,9.0,31.0,9.0,27.0,17.0,41.0,42.0,21.0,6.0,3.0,64.0,10.0,8.0,18.0,180.0,254.0,99.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,791.0,793.0,6.0,5.0,87.0,13.0,95.0,5.0,0.0,31.0,77.0,64.0,27.0,88.0,12.0,5.0,1.0,4.0,12.0,19.0,5.0,18.0,53.0,24.0,0.0,1.0,0.0,0.0,0.0,9.0,58.0,98.0,0.0,0.0,88.0,61.0,65.0,14.0,4.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,69.0,77.0,85.0,85.0,3360.0,201.0,618.0,630.0,658.0,600.0,643.0,21409.0,9.0,7.0,10.0,11.0,29.0,29.0,3.0,3.0,0.0,5.0,3.0,13.0,14.0,23.0,35.0,4.0,3.0,0.0,32.0,0.0,63.0,3.0,40.0,60.0,7.0,72.0,15.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,0.0,23.0,26.0,0.0,92.0,64.0,68.0,61.0,68.0,59.0,56.0,67.0,99.0,0.0,0.0,29.0,25.0,2.0,11.0,18.0,1.0,0.0,0.0,0.0,10.0,4.0,0.0,0.0,0.0,5.0,8.0,8.0,0.0,6.0,3.0,6.0,13.0,10.0,1.0,0.0,11.0,13.0,16.0,0.0,0.0,8.0,0.0,7.0,62.0,24.0,0.0,160.0,3.0,6.0,18.0,16.0,5.0,38.0,13.0,12.0,14.0,0.0,14.0,11.0,0.0,0.0,0.0,18.0,37.0,3.0,32.0,14.0,45.0,12.0,1.0,9.0,0.0,12.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,10.0,50.0,84.0,5.0,11.0,0.0,96.0,85.0,10.0,12.0,35.0,0.0,0.0,0.0,0.0,13.0,87.0,0.0,0.0,90.0,0.0,10.0,0.0,0.0,0.0,99.0,0.0,99.0,99.0,97.0,7.0,3.0,5.0,6.0,9706.0,9606.0,9604.0,9604.0,9603.0,9602.0,9601.0,9511.0,9510.0,9510.0,9508.0,9507.0,9506.0,9504.0,9503.0,9502.0,9501.0,9411.0,9411.0,9410.0,9409.0,9407.0,9406.0,23.0,9702.0,60.0,5.0,12.0,9607.0,9605.0,9607.0,9603.0,9603.0,9602.0,9512.0,9511.0,9511.0,9509.0,9507.0,9506.0,9505.0,9504.0,9503.0,9501.0,9412.0,9412.0,9411.0,9409.0,9408.0,9407.0,10.0,10.0,12.0,12.0,15.0,15.0,14.0,14.0,12.0,10.0,20.0,10.0,10.0,20.0,10.0,20.0,10.0,12.0,10.0,10.0,10.0,10.0,273.00,12.0,6.0,10.0,9509.0,30.0,9010.0,30.0,9702.0,8910.0,9001.0,3.0,22.750000,1.0,0.0,3.0,2.0
1,9001.0,0.0,2609.0,28.0,71.0,1.0,7.0,6.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,6.0,5.0,2.0,1335.0,372.0,424.0,99.0,0.0,0.0,51.0,49.0,92.0,1.0,0.0,3.0,10.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,3.0,31.0,37.0,40.0,32.0,41.0,44.0,26.0,35.0,43.0,22.0,13.0,28.0,24.0,15.0,12.0,5.0,3.0,15.0,13.0,36.0,22.0,14.0,13.0,2.0,10.0,7.0,31.0,63.0,39.0,13.0,4.0,63.0,8.0,3.0,26.0,253.0,314.0,99.0,99.0,0

In [41]:
X_train,X_test,y_train ,y_test = train_test_split(df,y,test_size=0.2,random_state = 42)
X_train,X_test ,y_train ,y_test

print(X_train_pre.shape)
# print(X_train_pre.info())
# X_train = df.select_dtypes(include=['float64','int64'])
print(X_train)

pca = PCA(n_components = 20)
X_train_after_pca = pca.fit_transform(X_train)
X_test_after_pca = pca.transform(X_test)


(95412, 411)
            0     1       2     3     4    5    6    7     8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24    25    26    27    28   29   30   31    32   33   34   35       36      37      38    39    40    41    42    43    44    45   46   47    48   49   50   51   52   53   54   55    56   57   58   59    60    61    62    63    64    65    66    67    68    69    70    71    72    73    74    75    76    77    78    79    80    81    82    83    84    85    86    87    88    89    90    91    92    93    94     95     96    97    98   99   100   101   102   103   104  105     106     107  108  109   110   111   112   113   114   115   116   117   118   119   120   121  122   123   124   125  126   127   128   129   130   131   132   133   134   135   136   137   138   139   140   141   142   143  144   145   146   147   148  149   150  151   152   153   154   155     156    157    158    159    160    161    162      163   164   165   1

(76329, 20)

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=15)
forest_clf.fit(X_train_after_pca, y_train)

#y_pred = model.predict(X_test_sub)
y_test_pred = cross_val_predict(forest_clf,X_test_after_pca, y_test, cv=3)
conf_mat = confusion_matrix(y_test,y_test_pred)
f1 = f1_score(y_test,y_test_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(forest_clf.score(X_test_sub, y_test)))
print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
display(conf_mat)